#Mount Google Drive

This will allow the Colab machine to access Google Drive folders by mounting the drive on the machine. You will be asked to copy and paste an authentication code.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Change directory to allow imports


In [2]:
import os
os.chdir("/content/gdrive/My Drive/CMPT 419 720 Project/")
!ls

'=2.0.1'				      poster.gdoc
 BoxDist24_GT				      References
 BoxDist24_TrackRCNN_Training		      sceneflow
 BoxDist31_GT				     'synthetic data'
 BoxDist31_TrackRCNN_Training		      testing
 cmpt419_mono_dept_esitmation_poster_v5.pdf   trackr-cnn
'CMPT419 Project Report.gdoc'		      TrackRCNN
 FairMOT				      TrackR-CNN
 FairMOT.ipynb				      trackr-cnn-v2
 MDE_Experiments			     'TrackR-CNN x BoxDist24'
'Milestone Report 1.gdoc'		     'TrackR-CNN x BoxDist31'
'Milestone Report 2.gdoc'		      training
 MonoDepthEstimation_v0.1		     'Weekly Briefings'
 output


##Import Libraries

In [3]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

import pycocotools

##Set Directory

In [4]:
BASE_DIR = '/content/gdrive/My Drive/CMPT 419 720 Project/'
TRAINING_DIR = os.path.join(BASE_DIR, 'training')
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

#Load & Process Data

In [5]:
set_names = ['0000', '0001', '0003', '0004', '0005', '0009', '0011', '0012', '0015', '0017', '0019', '0020']

In [6]:
import math

def dist(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    return math.sqrt(x**2+y**2+z**2)

##predict displacement with forward euler
def dynamic_disp(v1, v2, a1, a2, t=0.1, h=0.001):
  ad = (a2-a1)/t
  d = 0
  v = v1
  a = a1
  for t in range(int(t/h)):
    a = a + h*ad
    v = v + h*a
    d += v*h + 0.5*a*h**2
  return d

##Load Test GT Labels (Expected Outputs)

In [7]:
def load_gt_dists(set_names):
  annotations = {}
  kitti_anno_dir = os.path.join(TRAINING_DIR, 'label_02')
  tracking_anno_dir = os.path.join(TRAINING_DIR, 'tracking_data/segmentations')

  for set_name in set_names:
    kitti_anno_file = os.path.join(kitti_anno_dir, set_name+'.txt')
    tracking_anno_file = os.path.join(tracking_anno_dir, set_name+'.txt')
    records = {}

    tracking_anno = open(tracking_anno_file, 'r')
    tracking_lines = tracking_anno.readlines()
    line = tracking_lines[0]
    e = line.split(' ')
    height = int(e[3])
    width = int(e[4])

#Load Kitti Annotations
    kitti_anno = open(kitti_anno_file, 'r')
    kitti_lines = kitti_anno.readlines()
    for line in kitti_lines:
      e = line.split(' ')
      class_name = e[2]
      trunc = float(e[3])
      if (class_name != 'Car' and class_name != 'Pedestrian') or trunc > 0: #data exclusion
        continue
      frame = int(e[0])
      obj_id = int(e[1])
      if frame not in records.keys():
        records[frame] = {}
      if obj_id not in records[frame].keys():
        records[frame][obj_id] = {}
      records[frame][obj_id] = {
          'bbox': [float(e[6])/width, float(e[7])/height, float(e[8])/width, float(e[9])/height],
          'target': torch.from_numpy(np.array([float(e[13]), float(e[14]), float(e[15])])),
          'centre': [((float(e[6])+(float(e[8])-float(e[6]))/2)-(width/2))/width, ((float(e[7])+(float(e[9])-float(e[7]))/2)-(height/2))/height]
      }
    annotations[set_name] = records
  return annotations

In [8]:
gt_obj_dists = load_gt_dists(set_names)

##Load Track-RCNN Detections & Frame IMU Data (Inputs)

In [9]:
from pycocotools import mask as cocomask

def load_track_imu(set_names):
  annotations = {}
  tracking_anno_dir = os.path.join(TRAINING_DIR, 'tracking_data/segmentations')
  detection_anno_dir = os.path.join(TRAINING_DIR, 'tracking_data/detections')
  gps_anno_dir = os.path.join(TRAINING_DIR, 'oxts')

  for set_name in set_names:
    tracking_anno_file = os.path.join(tracking_anno_dir, set_name+'.txt')
    detection_anno_file = os.path.join(detection_anno_dir, set_name+'.txt')
    gps_anno_file = os.path.join(gps_anno_dir, set_name+'.txt')
    records = {}

#Load GPS Annotations
    gps_anno = open(gps_anno_file, 'r')
    gps_lines = gps_anno.readlines()
    gps = {}
    for frame, line in enumerate(gps_lines):
      e = line.split(' ')
      v = [float(e[8]), float(e[9]), float(e[10])]
      a = [float(e[11]), float(e[12]), float(e[13])]
      gps[frame] = {
          'v': v,
          'a': a,
      }
    
    tracking_anno = open(tracking_anno_file, 'r')
    tracking_lines = tracking_anno.readlines()
    for line in tracking_lines:
      e = line.split(' ')
      frame = int(e[0])
      if frame not in gps.keys():
        continue
      class_id = int(e[2])
      if class_id == 1:
        class_id = 0
      elif class_id == 2:
        class_id = 3
      obj_id = int(e[1])-1
      if frame not in records.keys():
        records[frame] = {}
        records[frame]['objs'] = {}
      height = int(e[3])
      width = int(e[4])
      records[frame]['height'] = height
      records[frame]['width'] = width
      records[frame]['v'] = gps[frame]['v']
      records[frame]['a'] = gps[frame]['a']
      records[frame]['frame'] = frame
      rle_obj = {
          'counts': e[5],
          'size': [height, width]
      }
      #bbox = cocomask.toBbox(rle_obj) #outputs XYWH
      if obj_id not in records[frame]['objs'].keys():
        records[frame]['objs'][obj_id] = {}
      #records[frame]['objs'][obj_id]['bbox'] = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
      records[frame]['objs'][obj_id]['rle'] = e[5].rstrip()
      records[frame]['objs'][obj_id]['class_id'] = class_id

    detection_anno = open(detection_anno_file, 'r')
    detection_lines = detection_anno.readlines()
    for line in detection_lines:
      e = line.split(' ')
      frame = int(e[0])
      confidence = float(e[5])
      if frame not in records.keys():
        continue
      bbox = [float(e[1]), float(e[2]), float(e[3]), float(e[4])]
      rle = e[9].rstrip()
      for obj_id in records[frame]['objs'].keys():
        if records[frame]['objs'][obj_id]['rle'] == rle:
          records[frame]['objs'][obj_id]['bbox'] = bbox
          break

    annotations[set_name] = records
  return annotations


In [10]:
tracking_imu_data = load_track_imu(set_names)

##Construct Model Inputs


In [11]:
def make_input(frame1, frame2, delta=1):
  height = frame1['height']
  width = frame1['width']
  img_center = [width/2, height/2]
  bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
  bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
  xd = -dynamic_disp(frame1['v'][1], frame2['v'][1], frame1['a'][1], frame2['a'][1], delta*0.1)
  yd = -dynamic_disp(frame1['v'][2], frame2['v'][2], frame1['a'][2], frame2['a'][2], delta*0.1)
  zd = dynamic_disp(frame1['v'][0], frame2['v'][0], frame1['a'][0], frame2['a'][0], delta*0.1)
  disp = [xd, yd, zd]
  bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
  bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
  w1 = bbox1[2] - bbox1[0]
  w2 = bbox2[2] - bbox2[0]
  h1 = bbox1[3] - bbox1[1]
  h2 = bbox2[3] - bbox2[1]
  a1 = w1*h1
  a2 = w2*h2
  c1 = [(frame1['bbox'][0]+(w1/2)-img_center[0])/(width/2), (frame1['bbox'][1]+(h1/2)-img_center[1])/(height/2)]
  c2 = [(frame2['bbox'][0]+(w2/2)-img_center[0])/(width/2), (frame2['bbox'][1]+(h2/2)-img_center[1])/(height/2)]
  class_id = frame1['class_id']
  id_vec = [0, 0, 0, 0, 0, 0, 0]
  id_vec[class_id] = 1
  input = torch.from_numpy(np.array(id_vec + bbox1 + bbox2 + disp + [w1, w2, h1, h2, a1, a2] + c1 + c2 + [w2/w1, h2/h1, a2/a1]).astype(np.float32)).cuda()
  return input

##Construct Inputs & Targets

In [12]:
def bbox_overlap_ratio(bbox0, bboxs):
  ratios = []
  area0 = (bbox0[2]-bbox0[0]) * (bbox0[3]-bbox0[1])
  for bbox in bboxs:
    x_left = max(bbox0[0], bbox[0])
    y_top = max(bbox0[1], bbox[1])
    x_right = min(bbox0[2], bbox[2])
    y_bottom = min(bbox0[3], bbox[3])
    if x_right < x_left or y_bottom < y_top:
      iou = 0.0
      ratios.append(iou)
    else:
      intersection_area = (x_right - x_left) * (y_bottom - y_top)
      bb1_area = (bbox0[2] - bbox0[0]) * (bbox0[3] - bbox0[1])
      bb2_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
      iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
      ratios.append(iou)

  return ratios

def align_input_to_gt(inputs, gts):
  input_bboxs = []
  gt_bboxs = []
  for input in inputs:
    numpy_input = input.detach().cpu().numpy()
    input_bbox = [numpy_input[11], numpy_input[12], numpy_input[13], numpy_input[14]]
    input_bboxs.append(input_bbox)
  for obj_id in gts.keys():
    gt_bbox = gts[obj_id]['bbox']
    gt_bboxs.append(gt_bbox)
  
  ratio_table = []
  for gt_bbox in gt_bboxs:
    ratios = bbox_overlap_ratio(gt_bbox, input_bboxs)
    ratio_table.append(ratios)
  ratio_table = np.array(ratio_table)

  alignments = [-1]*len(gts)

  gt_maxs = []
  for i in range(len(gts)):
    gt_maxs.append(np.argmax(ratio_table[i, :]))
  input_maxs = []
  for i in range(len(inputs)):
    input_maxs.append(np.argmax(ratio_table[:, i]))

  for me, want in enumerate(gt_maxs):
    if input_maxs[want] == me and ratio_table[me, want] > 0.0:
      alignments[me] = want

  return alignments

In [13]:
master_inputs = []
master_targets = []
for set_name in set_names:
  prev_data = {}
  for frame in gt_obj_dists[set_name].keys():
    if frame not in tracking_imu_data[set_name].keys():
      continue
    inputs = []
    for obj_id in tracking_imu_data[set_name][frame]['objs']:
      this_frame = tracking_imu_data[set_name][frame]
      if obj_id in prev_data.keys() and 'bbox' in this_frame['objs'][obj_id].keys():
        last_frame = prev_data[obj_id]
        if 'bbox' in last_frame['objs'][obj_id].keys():
          frame1 = {
              'v': last_frame['v'],
              'a': last_frame['a'],
              'bbox': last_frame['objs'][obj_id]['bbox'],
              'class_id': last_frame['objs'][obj_id]['class_id'],
              'height': last_frame['height'], 
              'width': last_frame['width'],
          }
          frame2 = {
              'v': this_frame['v'],
              'a': this_frame['a'],
              'bbox': this_frame['objs'][obj_id]['bbox'],
              'class_id': this_frame['objs'][obj_id]['class_id'],
          }
          delta = this_frame['frame'] - last_frame['frame']
          input = make_input(frame1, frame2, delta)
          inputs.append(input)
      prev_data[obj_id] = this_frame

    if len(inputs) == 0:
      continue

    frame_gt = gt_obj_dists[set_name][frame]

    alignment = align_input_to_gt(inputs, frame_gt)

    targets = []

    for obj_id in frame_gt.keys():
      targets.append(frame_gt[obj_id]['target'])
    
    for t, o in enumerate(alignment):
      if o == -1:
        continue
      master_inputs.append(inputs[o])
      master_targets.append(targets[t])

#Training

##Network

In [14]:
n_epochs = 60
learning_rate = 0.001
batch_size = 64

In [16]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class BBoxDistNet(nn.Module):
  def __init__(self):
    super(BBoxDistNet, self).__init__()

    self.fc_net = nn.Sequential(
      nn.Linear(31, 18),
      nn.LayerNorm(18),
      nn.LeakyReLU(0.01, inplace=True),
      nn.Linear(18, 18),
      nn.LayerNorm(18),
      nn.LeakyReLU(0.01, inplace=True),
      nn.Linear(18, 3),
    )
  def forward(self, x):
    x = self.fc_net(x)
    return x

net = BBoxDistNet().cuda()

optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0)
criterion = nn.L1Loss()

##Train

In [15]:
print(len(master_inputs))
print(len(master_targets))

15786
15786


In [17]:
for epoch in range(n_epochs):
  net.train()
  running_loss = 0
  for i in range(len(master_inputs)):
    input = master_inputs[i].cuda()
    output = net(input)
    target = master_targets[i].cuda()
    optimizer.zero_grad()
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
  running_loss/=len(master_inputs)
  print('[%d] training_loss: %.3f' %(epoch + 1, running_loss))

[1] training_loss: 1.676
[2] training_loss: 0.788
[3] training_loss: 0.741
[4] training_loss: 0.702
[5] training_loss: 0.700
[6] training_loss: 0.691
[7] training_loss: 0.682
[8] training_loss: 0.683
[9] training_loss: 0.683
[10] training_loss: 0.676
[11] training_loss: 0.669
[12] training_loss: 0.664
[13] training_loss: 0.663
[14] training_loss: 0.667
[15] training_loss: 0.662
[16] training_loss: 0.659
[17] training_loss: 0.655
[18] training_loss: 0.649
[19] training_loss: 0.655
[20] training_loss: 0.647
[21] training_loss: 0.646
[22] training_loss: 0.646
[23] training_loss: 0.652
[24] training_loss: 0.649
[25] training_loss: 0.649
[26] training_loss: 0.647
[27] training_loss: 0.647
[28] training_loss: 0.647
[29] training_loss: 0.645
[30] training_loss: 0.647
[31] training_loss: 0.646
[32] training_loss: 0.646
[33] training_loss: 0.648
[34] training_loss: 0.639
[35] training_loss: 0.633
[36] training_loss: 0.631
[37] training_loss: 0.634
[38] training_loss: 0.625
[39] training_loss: 0

In [18]:
torch.save(net, os.path.join(OUTPUT_DIR, 'bbox31_trackrcnn.pt'))

#Video Output

In [ ]:
import pycocotools
from skimage import measure

class_ids = {
    'Car': 0,
    'Van': 1,
    'Truck': 2,
    'Pedestrian': 3,
    'Person': 4,
    'Cyclist': 5,
    'Tram': 6,
}
def frame_first_set_annotations(set_name):
  kitti_anno_dir = os.path.join(TRAINING_DIR, 'label_02')
  tracking_anno_dir = os.path.join(TRAINING_DIR, 'instances_txt')
  tf_anno_dir = os.path.join(TRAINING_DIR, 'instances')
  gps_anno_dir = os.path.join(TRAINING_DIR, 'oxts')

  kitti_anno_file = os.path.join(kitti_anno_dir, set_name+'.txt')
  tracking_anno_file = os.path.join(tracking_anno_dir, set_name+'.txt')
  gps_anno_file = os.path.join(gps_anno_dir, set_name+'.txt')
  records = {}

  tracking_anno = open(tracking_anno_file, 'r')
  tracking_lines = tracking_anno.readlines()
  line = tracking_lines[0]
  e = line.split(' ')
  frame = int(e[0])
  height = int(e[3])
  width = int(e[4])

#Load Kitti Annotations
  kitti_anno = open(kitti_anno_file, 'r')
  kitti_lines = kitti_anno.readlines()
  for line in kitti_lines:
    e = line.split(' ')
    class_name = e[2]
    trunc = float(e[3])
    frame = int(e[0])
    if frame not in records.keys():
      records[frame] = {}
      records[frame]['objs'] = {}
    if class_name == 'DontCare' or class_name == 'Misc' or trunc > 0.0: #data exclusion
      continue
    obj_id = int(e[1])
    records[frame]['img_id'] = set_name+str(frame)
    records[frame]['objs'][obj_id] = {
        'class_name': class_name,
        'class_id': class_ids[class_name],
        'alpha': float(e[5]),
        'bbox': [float(e[6]), float(e[7]), float(e[8]), float(e[9])],
        'xyz': [float(e[13]), float(e[14]), float(e[15])],
        'wlht': [float(e[10]), float(e[11]), float(e[12]), float(e[16])],
        'height': height,
        'width': width,
    }

#Load GPS Annotations
  gps_anno = open(gps_anno_file, 'r')
  gps_lines = gps_anno.readlines()
  for frame, line in enumerate(gps_lines):
    e = line.split(' ')
    if frame not in records.keys():
      records[frame] = {}
    v = [float(e[8]), float(e[9]), float(e[10])]
    a = [float(e[11]), float(e[12]), float(e[13])]
    records[frame]['dynamics'] = {
        'v': v,
        'a': a,
    }

  return records

In [ ]:
set_names = ['0020']
for set_name in set_names:
  annotations = frame_first_set_annotations(set_name)
  make_videos(set_name)

000000
(376, 1241, 3)
000001
(376, 1241, 3)
000002
(376, 1241, 3)
000003
(376, 1241, 3)
000004
(376, 1241, 3)
000005
(376, 1241, 3)
000006
(376, 1241, 3)
000007
(376, 1241, 3)
000008
(376, 1241, 3)
000009
(376, 1241, 3)
000010
(376, 1241, 3)
000011
(376, 1241, 3)
000012
(376, 1241, 3)
000013
(376, 1241, 3)
000014
(376, 1241, 3)
000015
(376, 1241, 3)
000016
(376, 1241, 3)
000017
(376, 1241, 3)
000018
(376, 1241, 3)
000019
(376, 1241, 3)
000020
(376, 1241, 3)
000021
(376, 1241, 3)
000022
(376, 1241, 3)
000023
(376, 1241, 3)
000024
(376, 1241, 3)
000025
(376, 1241, 3)
000026
(376, 1241, 3)
000027
(376, 1241, 3)
000028
(376, 1241, 3)
000029
(376, 1241, 3)
000030
(376, 1241, 3)
000031
(376, 1241, 3)
000032
(376, 1241, 3)
000033
(376, 1241, 3)
000034
(376, 1241, 3)
000035
(376, 1241, 3)
000036
(376, 1241, 3)
000037
(376, 1241, 3)
000038
(376, 1241, 3)
000039
(376, 1241, 3)
000040
(376, 1241, 3)
000041
(376, 1241, 3)
000042
(376, 1241, 3)
000043
(376, 1241, 3)
000044
(376, 1241, 3)
000045
(37

In [ ]:
net = torch.load(os.path.join(OUTPUT_DIR, 'bbox_multi-multi_trunc_class_61.pt'))

In [ ]:
def make_videos(set_name):
  img_dir = os.path.join(os.path.join(TRAINING_DIR, 'image_02'), set_name)
  height = 376
  width = 1241
  img_center = [width/2, height/2]
  vid_writer = cv2.VideoWriter(filename=OUTPUT_DIR+"/dist_est_demo_multi_"+set_name+".avi",  #Provide a file to write the video to
  fourcc=cv2.VideoWriter_fourcc('p', 'n', 'g', ' '),            #Use whichever codec works for you...
  fps=10,                                        #How many frames do you want to display per second in your video?
  frameSize=(width, height))   

  #prev data
  data = {}

  net.eval()
  with torch.no_grad():
      for frame in annotations.keys():
        img_name = '000000'
        img_name = img_name[0:6-len(str(frame))] + str(frame)
        print(img_name)
        img = cv2.imread(os.path.join(img_dir, img_name+'.png'))
        print(img.shape)
        frame_anno = annotations[frame]
        if 'objs' not in frame_anno.keys():
          continue
        for obj_id in frame_anno['objs'].keys():
          obj_anno = frame_anno['objs'][obj_id]
          class_name = obj_anno['class_name']
          #maybe do exclusion
          #if class_name != 'Car':
          #  continue
          bbox = obj_anno['bbox'] #XYXY_ABS
          dynamics = frame_anno['dynamics']
          if obj_id not in data.keys():
            data[obj_id] = {}
          data[obj_id][frame] = {
              'bbox': bbox,
              'v': dynamics['v'],
              'a': dynamics['a'],
              'class_id': obj_anno['class_id'],
              'gt': dist(obj_anno['xyz'])
          }
          #continue due to no prev data
          if frame-1 not in data[obj_id].keys():
            continue
          #feed into model
          frame1 = data[obj_id][frame-1]
          frame2 = data[obj_id][frame]
          bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
          bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
          xd = -dynamic_disp(frame1['v'][1], frame2['v'][1], frame1['a'][1], frame2['a'][1])
          yd = -dynamic_disp(frame1['v'][2], frame2['v'][2], frame1['a'][2], frame2['a'][2])
          zd = dynamic_disp(frame1['v'][0], frame2['v'][0], frame1['a'][0], frame2['a'][0])
          disp = [xd, yd, zd]
          bbox1 = [frame1['bbox'][0]/width, frame1['bbox'][1]/height, frame1['bbox'][2]/width, frame1['bbox'][3]/height]
          bbox2 = [frame2['bbox'][0]/width, frame2['bbox'][1]/height, frame2['bbox'][2]/width, frame2['bbox'][3]/height]
          w1 = bbox1[2] - bbox1[0]
          w2 = bbox2[2] - bbox2[0]
          h1 = bbox1[3] - bbox1[1]
          h2 = bbox2[3] - bbox2[1]
          a1 = w1*h1
          a2 = w2*h2
          c1 = [(frame1['bbox'][0]+(w1/2)-img_center[0])/(width/2), (frame1['bbox'][1]+(h1/2)-img_center[1])/(height/2)]
          c2 = [(frame2['bbox'][0]+(w2/2)-img_center[0])/(width/2), (frame2['bbox'][1]+(h2/2)-img_center[1])/(height/2)]
          class_id = frame1['class_id']
          id_vec = [0, 0, 0, 0, 0, 0, 0]
          id_vec[class_id] = 1
          input = np.array(id_vec + bbox1 + bbox2 + disp + [w1, w2, h1, h2, a1, a2] + c1 + c2 + [w2/w1, h2/h1, a2/a1]).astype(np.float32)
          input = torch.from_numpy(input).cuda()
          dist_estimate = dist(net(input))
          #draw boxes
          label = '%.2f | %.2f' % (dist_estimate, frame2['gt'])
          x1 = int(bbox[0])
          x2 = int(bbox[2])
          y1 = int(bbox[1])
          y2 = int(bbox[3])
          cv2.rectangle(img,(x1,y1),(x2,y2),(0,200,0),1)
          labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_SIMPLEX,0.5,2)
          _x1 = x1
          _y1 = y1#+int(labelSize[0][1]/2)
          _x2 = _x1+labelSize[0][0]
          _y2 = y1-int(labelSize[0][1])
          cv2.rectangle(img,(_x1,_y1),(_x2,_y2),(0,200,0),cv2.FILLED)
          cv2.putText(img,label,(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,255),1)
          #write to video
        vid_writer.write(img)

  vid_writer.release()

In [ ]:
vid_writer.release()

NameError: ignored

In [ ]:
!ls output/

bbox_car_107_trunc.pt		    bbox_multi-multi_trunc_class_85.pt
bbox_multi_116_trunc.pt		    bbox_sans_car_137.pt
bbox_multi_multi_136.pt		    dist_est_demo_multi_0000.avi
bbox_multi-multi_trunc_class_61.pt  dist_est_demo_multi_0001.avi
